In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import pandas as pd
import time

In [2]:
#Setting up the browser
executable_path={'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
### NASA Mars News
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
tlts=[]
bdys=[]
html = browser.html
soup = bs(html, 'html.parser')

results = soup.find_all(class_='slide')
print("---Scraping Begins---")
for result in results[:20]:
    tlt = result.find(class_='content_title').text.strip()
    bdy = result.find(class_='article_teaser_body').text.strip()
    tlts.append(tlt)
    bdys.append(bdy)
    #print(tlt)
    #print(bdy)
    #print("---scraping---")
try:
    #browser.click_link_by_partial_text('more')
    browser.links.find_by_partial_text('more')
except:
    print("---Scraping Complete---")

---Scraping Begins---


In [4]:
# new title and details
news_title=tlts[0]
news_p=bdys[0]
print(news_title)
print(news_p)

Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover
NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries.


In [5]:
### JPL Mars Space Images - Featured Image ---
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)

#Find the full_image id and clicking on it 
full_image_click = browser.find_by_id('full_image')
full_image_click.click()

#Setting a wait time before clicking on more info
more_info = browser.is_element_present_by_text('more info', wait_time=10)
more_info = browser.links.find_by_partial_text('more info')
more_info.click()


#Using Beautiful Soup to parse the html
image_html = browser.html
image_soup = bs(image_html, 'html.parser')

#Selecting html tag
featured_image = image_soup.select_one('figure.lede a img').get('src')

#Making a long url
featured_image_url = 'https://www.jpl.nasa.gov' + featured_image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA20316_hires.jpg


In [6]:
### Mars Weather ---
url3 = 'https://twitter.com/marswxreport?lang=en'
response_twtr = requests.get(url3)
soup = bs(response_twtr.text, 'html.parser')
mars_weathers = soup.find_all('p', class_="TweetTextSize")
for tweet in mars_weathers:
    tweet.find('a').extract()
    if 'InSight sol' in tweet.text:
        mars_weather = tweet.text
        break

print(f'Mars weather:\n{mars_weather}')

Mars weather:
InSight sol 457 (2020-03-10) low -95.7ºC (-140.3ºF) high -9.1ºC (15.6ºF)
winds from the SSE at 6.5 m/s (14.5 mph) gusting to 21.0 m/s (46.9 mph)
pressure at 6.30 hPa


In [8]:
### Mars Facts ---
url4='https://space-facts.com/mars/'
table = pd.read_html(url4)
mars_facts = table[0]
mars_fact = []
for i in range(len(mars_facts[0])):
    mars_fact.append([mars_facts[0][i], mars_facts[1][i]])
mars_fact

[['Equatorial Diameter:', '6,792 km'],
 ['Polar Diameter:', '6,752 km'],
 ['Mass:', '6.39 × 10^23 kg (0.11 Earths)'],
 ['Moons:', '2 (Phobos & Deimos)'],
 ['Orbit Distance:', '227,943,824 km (1.38 AU)'],
 ['Orbit Period:', '687 days (1.9 years)'],
 ['Surface Temperature:', '-87 to -5 °C'],
 ['First Record:', '2nd millennium BC'],
 ['Recorded By:', 'Egyptian astronomers']]

In [9]:
### Mars hemispheres ---
url4= 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url4)
soup = bs(browser.html, 'html.parser')

hemispheres_image_urls = []

results_hemis = soup.find_all('div', class_='item')

for item in results_hemis:
    title_hemis = item.h3.text
    first_url = url4[:30] + item.find('a', class_='itemLink')['href']
    browser.visit(first_url)
    soup = bs(browser.html, 'html.parser')
    time.sleep(1)
    final_url = url4[:30] + soup.find('img', class_='wide-image')['src']
    hemispheres_image_urls .append({'title': title_hemis, 'img_url':final_url})
    
hemispheres_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]